# Финальный проект. SQL

**Описание проекта**

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. 

**Цель исследования:** проанализировать базу данных, в которой хранится информация о книгах, издательствах и авторах, выявить особенности и закономерности, которые помогут внести конкурентное преимущество в новый продукт.

**Описание данных**

Таблица `books`

Содержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Таблица `authors`

Содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

Таблица `publishers`

Содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

Таблица `ratings`

Содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Таблица `reviews`

Содержит данные о пользовательских обзорах:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.


In [1]:
import pandas as pd
from sqlalchemy import create_engine 


## Обзор данных

In [2]:
db_config = {'user': 'praktikum_student', 
             'pwd': 'Sdf4$2;d-d30pp', 
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, 
             'db': 'data-analyst-final-project-db'} 
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

In [3]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [4]:
query = '''
           SELECT *
           FROM books
        '''

books = pd.io.sql.read_sql(query, con=engine)

books.head(5) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
query = '''
           SELECT *
           FROM authors
        '''

authors = pd.io.sql.read_sql(query, con=engine)

authors.head(5) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
query = '''
           SELECT *
           FROM publishers
        '''

publishers = pd.io.sql.read_sql(query, con=engine)

publishers.head(5) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [15]:
query = '''
           SELECT *
           FROM ratings
        '''

ratings = pd.io.sql.read_sql(query, con=engine)

ratings.head(5) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [8]:
query = '''
           SELECT *
           FROM reviews
        '''

reviews = pd.io.sql.read_sql(query, con=engine)

reviews.head(5) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Задания

###  Посчитайте, сколько книг вышло после 1 января 2000 года

In [9]:
query = '''
           SELECT COUNT(DISTINCT(book_id))
           FROM books
           WHERE publication_date > '2000-01-01'
        '''
pd.io.sql.read_sql(query, con = engine) 

,count
0,819


С первого января 2000 года вышло 819 книг.

### Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [55]:
query = '''
        WITH ratings_stats AS (SELECT b.title as title, 
                                      b.book_id as book_id, 
                                      AVG(ret.rating) as avg_rating
                               FROM books as b        
                               LEFT JOIN ratings as ret ON b.book_id = ret.book_id
                               GROUP BY b.book_id),
        reveiws_stats AS (SELECT b.title, 
                                 b.book_id, 
                                 COUNT(rev.book_id) as count_reviews
                          FROM books as b
                          LEFT JOIN reviews as rev ON b.book_id = rev.book_id
                          GROUP BY b.book_id)
                          
        SELECT ratings_stats.book_id, 
               ratings_stats.title, 
               ROUND(ratings_stats.avg_rating, 1), 
               reveiws_stats.count_reviews
        FROM ratings_stats
        LEFT JOIN reveiws_stats ON ratings_stats.book_id = reveiws_stats.book_id
        '''
pd.io.sql.read_sql(query, con = engine) 

,book_id,title,round,count_reviews
0,1,'Salem's Lot,3.7,2
1,2,1 000 Places to See Before You Die,2.5,1
2,3,13 Little Blue Envelopes (Little Blue Envelope...,4.7,3
3,4,1491: New Revelations of the Americas Before C...,4.5,2
4,5,1776,4.0,4
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3.7,3
996,997,Xenocide (Ender's Saga #3),3.4,3
997,998,Year of Wonders,3.2,4
998,999,You Suck (A Love Story #2),4.5,2


Для каждой книги мы получили среднюю оценку и количество обзоров.

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [56]:
query = '''
           SELECT p.publisher, 
                  COUNT(b.book_id) as total_books
           FROM books AS b
           LEFT JOIN publishers AS p ON p.publisher_id = b.publisher_id
           WHERE num_pages > 50
           GROUP BY p.publisher
           ORDER BY total_books DESC
           LIMIT 1
           
        '''
pd.io.sql.read_sql(query, con = engine) 

,publisher,total_books
0,Penguin Books,42


Издательство, выпустичшее наибольшее число книг, а именно 42 книги, называется «Penguin Books».

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [57]:
query = '''
           SELECT a.author, 
                  AVG(rat.rating) as avg_rat
           FROM authors AS a
           LEFT JOIN books AS b ON a.author_id = b.author_id
           LEFT JOIN ratings AS rat ON b.book_id=rat.book_id
           WHERE b.book_id IN (SELECT book_id 
                               FROM (SELECT b.book_id, COUNT(rat.rating_id) AS count_rat
                                     FROM books AS b
                                     LEFT JOIN ratings AS rat ON b.book_id=rat.book_id
                                     GROUP BY b.book_id
                                     ORDER BY count_rat DESC) as t1 
                                WHERE t1.count_rat > 50)
           GROUP BY a.author
           ORDER BY avg_rat DESC
           LIMIT 1
           
        '''
pd.io.sql.read_sql(query, con = engine) 

,author,avg_rat
0,J.K. Rowling/Mary GrandPré,4.287097


J.K. Rowling/Mary GrandPré являются авторами с наивысшей средней оценкой, среди книг, у которых больше 50 оценок.

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [58]:
query = '''
           SELECT AVG(count_review.count) as avg_review_count
           FROM (SELECT username, 
                        COUNT(review_id) AS count
                 FROM reviews
                 GROUP BY username) AS count_review
           WHERE count_review.username IN (SELECT username 
                                           FROM (SELECT username, COUNT(rating_id) AS count_rat
                                                 FROM ratings
                                                 GROUP BY username
                                                 ORDER BY count_rat DESC) as t1 
                                           WHERE t1.count_rat > 50)
           
        '''
pd.io.sql.read_sql(query, con = engine) 

,avg_review_count
0,24.333333


Среднее количество обзоров, среди пользователей поставивших больше 50 оценок - 24,3.

**Вывод:** с 2000 года было выпущено 819 книг, среди них были как очень успешные так и провальные - с этими данными можно ознакомиться в таблице с количеством обзоров и средним рейтингом для каждой книги. Самым продуктивным изданием оказалось Penguin Books - оно выпустило 42 книги. J.K. Rowling/Mary GrandPré являются авторами бестселлера и обладелями наивысшей средней оценки книг. Так же мы узнали что пользователи, часто оценивающие книги в среднем делают 24 обзора.